<a href="https://colab.research.google.com/github/imenFerjani/Road-Anomalies-Detection/blob/main/FishSchoolSeach_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################# Import Packages ###########################

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import string
import keras
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
plt.style.use('seaborn')
from copy import deepcopy,copy
import pandas as pd
import time
from tqdm import tqdm


import random
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, Embedding, SpatialDropout1D, Conv1D, MaxPool1D, Flatten, concatenate, Dense, Dropout

from keras.preprocessing import sequence
from keras.datasets import imdb

from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix



import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
plt.style.use('ggplot')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier


Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
# WORD-level classification
MAX_NUM_WORDS     = 5000
EMBEDDING_DIM     = 300
MAX_SEQ_LENGTH    = 512

# GENERAL
DROPOUT_RATE      = 0.5
#NB_CLASSES        = 1

# LEARNING
BATCH_SIZE        = 32
NB_EPOCHS         = 1
#RUNS              = 5


In [ ]:
# loading imdb database
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=MAX_NUM_WORDS)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')


print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=MAX_SEQ_LENGTH)
x_test = sequence.pad_sequences(x_test, maxlen=MAX_SEQ_LENGTH)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=MAX_SEQ_LENGTH)
x_test = sequence.pad_sequences(x_test, maxlen=MAX_SEQ_LENGTH)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 500)
x_test shape: (25000, 500)
Pad sequences (samples x time)
x_train shape: (25000, 500)
x_test shape: (25000, 500)


In [ ]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

NameError: ignored

In [ ]:
print(type(x_train))

<class 'numpy.ndarray'>


In [ ]:
x_train=np.concatenate((x_train , x_test[:10000,:]))
x_test = x_test[10000:,:]
y_train=np.concatenate((y_train , y_test[:10000]))
y_test = y_test[10000:]


In [ ]:

# loading reuters database

from keras.datasets import reuters

(x_train, y_train), (x_test, y_test) = reuters.load_data(path="reuters.npz",
                                                         num_words=MAX_NUM_WORDS,
                                                         skip_top=0,
                                                         maxlen=MAX_SEQ_LENGTH,
                                                         test_split=0.2,
                                                         seed=113,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)
word_index = reuters.get_word_index(path="reuters_word_index.json")

print('# of Training Samples: {}'.format(len(x_train)))
print('# of Test Samples: {}'.format(len(x_test)))

nb_classes = max(y_train) + 1
print('# of Classes: {}'.format(nb_classes))
index_to_word = {}
for key, value in word_index.items():
    index_to_word[value] = key
print(' '.join([index_to_word[x] for x in x_train[0]]))
print(y_train[0])

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

y_train = keras.utils.to_categorical(y_train, nb_classes)
y_test = keras.utils.to_categorical(y_test, nb_classes)


557056/550378 [==============================] - 0s 0us/step
# of Training Samples: 8384
# of Test Samples: 2097
# of Classes: 46
the of of mln loss for plc said at only ended said of could 1 traders now april 0 a after said from 1985 and from foreign 000 april 0 prices its account year a but in this mln home an states earlier and rise and revs vs 000 its 16 vs 000 a but 3 of of several and shareholders and dividend vs 000 its all 4 vs 000 1 mln agreed of april 0 are 2 states will billion total and against 000 pct dlrs
3


In [ ]:
# loading Yelp database
filepath_dict = {'yelp':'yelp_labelled.txt','amazon':'amazon_cells_labelled.txt','imdb':'imdb_labelled.txt'}
df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source  # Add another column filled with the source name
    df_list.append(df)
df = pd.concat(df_list)
df_yelp = df[df['source'] == 'yelp']
sentences = df_yelp['sentence'].values
y = df_yelp['label'].values
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)
encoder = LabelEncoder()
y_train= encoder.fit_transform(y_train)
y_test=encoder.fit_transform(y_test)

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

#vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

maxlen = MAX_SEQ_LENGTH #100

x_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(X_test, padding='post', maxlen=maxlen)



In [ ]:
# loading amazon database

filepath_dict = {'yelp':'yelp_labelled.txt','amazon':'amazon_cells_labelled.txt','imdb':'imdb_labelled.txt'}
df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source  # Add another column filled with the source name
    df_list.append(df)
df = pd.concat(df_list)
df_yelp = df[df['source'] == 'amazon']
sentences = df_yelp['sentence'].values
y = df_yelp['label'].values
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)
encoder = LabelEncoder()
y_train= encoder.fit_transform(y_train)
y_test=encoder.fit_transform(y_test)
#vectorizer = CountVectorizer()
#vectorizer.fit(sentences_train)

#X_train = vectorizer.transform(sentences_train)
#X_test  = vectorizer.transform(sentences_test)

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

#vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

maxlen = MAX_SEQ_LENGTH #100

x_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
# Yahoo Answers Dataset DATASET
train_data=pd.read_csv('trainyahoo.csv',header=None, names=['class_id','qtitle','question','best_answer'], error_bad_lines=False, engine='python', encoding='utf8')
test_data=pd.read_csv("testyahoo.csv",header=0, names=['class_id','qtitle','question','best_answer'], error_bad_lines=False, engine='python', encoding='utf8')
frames = [train_data, test_data]
dataset_agnews_fss = pd.concat(frames)
c1=dataset_agnews_fss.query('class_id == 1').sample(1000)
c2=dataset_agnews_fss.query('class_id == 2').sample(1000)
c3=dataset_agnews_fss.query('class_id == 3').sample(1000)
c4=dataset_agnews_fss.query('class_id == 4').sample(1000)
c5=dataset_agnews_fss.query('class_id == 5').sample(1000)
c6=dataset_agnews_fss.query('class_id == 6').sample(1000)
c7=dataset_agnews_fss.query('class_id == 7').sample(1000)
c8=dataset_agnews_fss.query('class_id == 8').sample(1000)
c9=dataset_agnews_fss.query('class_id == 9').sample(1000)
c10=dataset_agnews_fss.query('class_id == 10').sample(1000)
dataset_agnews_fss=pd.concat([c1,c2,c3,c4,c5,c6,c7,c8,c9,c10])
dataset_agnews_fss.best_answer=dataset_agnews_fss.best_answer.astype(str)
sentences_fss = dataset_agnews_fss['best_answer'].values
y_fss = dataset_agnews_fss['class_id'].values

sentences_train_fss, sentences_test_fss, y_train_fss, y_test_fss =train_test_split(sentences_fss, y_fss, test_size=0.25, random_state=1000)

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(sentences_train_fss)

X_train_fss = tokenizer.texts_to_sequences(sentences_train_fss)
X_test_fss = tokenizer.texts_to_sequences(sentences_test_fss)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
maxlen = MAX_SEQ_LENGTH

X_train_fss = pad_sequences(X_train_fss, padding='post', maxlen=maxlen)
X_test_fss = pad_sequences(X_test_fss, padding='post', maxlen=maxlen)

y_train_fss=[x-1 for x in y_train_fss]
y_test_fss=[x-1 for x in y_test_fss]
y_train_fss = keras.utils.to_categorical(y_train_fss, num_classes=10)
y_test_fss = keras.utils.to_categorical(y_test_fss, num_classes=10)

x_train,x_test,y_train,y_test = X_train_fss[:1000,:], X_test_fss[:1000,:], y_train_fss[:1000,:],y_test_fss[:1000,:]
#x_train,x_test,y_train,y_test = X_train_fss[:1000,:], X_test_fss[:1000,:], y_train_fss[:1000,:],y_test_fss[:1000,:]

Skipping line 1063032: unexpected end of data


In [ ]:
type(sentences_fss)

numpy.ndarray

In [ ]:
# loading Dataset
# word preprocessing dataset

train_data=pd.read_csv('/gdrive/My Drive/textdatasets/trainyahoo.csv',header=None, names=['classid','a','b','desc'], error_bad_lines=False, engine='python', encoding='utf8')
test_data=pd.read_csv('/gdrive/My Drive/textdatasets/testyahoo.csv',header=0, names=['classid','a','b','desc'], error_bad_lines=False, engine='python', encoding='utf8')
train_data.desc = train_data.desc.astype(str)
test_data.desc = test_data.desc.astype(str)
#train_data=pd.read_csv('train.csv', delimiter=',',header=0,names=['classid','title','desc'])
#test_data=pd.read_csv("test.csv",header=0,names=['classid','title','desc'])
frames = [train_data, test_data]

dataset_agnews_fss = pd.concat(frames)
#dataset_agnews_fss = dataset_agnews[:1000]
sentences_fss = dataset_agnews_fss['desc'].values
y_fss = dataset_agnews_fss['classid'].values

sentences_train_fss, sentences_test_fss, y_train_fss, y_test_fss =train_test_split(sentences_fss, y_fss, test_size=0.0428571429, random_state=1000)

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(sentences_train_fss)

X_train_fss = tokenizer.texts_to_sequences(sentences_train_fss)
X_test_fss = tokenizer.texts_to_sequences(sentences_test_fss)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
maxlen = MAX_SEQ_LENGTH

X_train_fss = pad_sequences(X_train_fss, padding='post', maxlen=maxlen)
X_test_fss = pad_sequences(X_test_fss, padding='post', maxlen=maxlen)

y_train_fss=[x-1 for x in y_train_fss]
y_test_fss=[x-1 for x in y_test_fss]
y_train_fss = keras.utils.to_categorical(y_train_fss, num_classes=10)
y_test_fss = keras.utils.to_categorical(y_test_fss, num_classes=10)

x_train,x_test,y_train,y_test = X_train_fss[:1000,:], X_test_fss[:1000,:], y_train_fss[:1000,:],y_test_fss[:1000,:]


In [ ]:
#Yelp FULL DATASET################################################################
train_data=pd.read_csv('trainamzF.csv',header=None, names=['class_id','title','desc'], error_bad_lines=False, engine='python', encoding='utf8')
test_data=pd.read_csv("testamzF.csv",header=0, names=['class_id','title','desc'], error_bad_lines=False, engine='python', encoding='utf8')

frames = [train_data, test_data]

dataset_agnews_fss = pd.concat(frames)
#dataset_agnews_fss = dataset_agnews[:1000]
sentences_fss = dataset_agnews_fss['desc'].values
y_fss = dataset_agnews_fss['class_id'].values

sentences_train_fss, sentences_test_fss, y_train_fss, y_test_fss =train_test_split(sentences_fss, y_fss, test_size=0.25, random_state=1000)

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(sentences_train_fss)

X_train_fss = tokenizer.texts_to_sequences(sentences_train_fss)
X_test_fss = tokenizer.texts_to_sequences(sentences_test_fss)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
maxlen = MAX_SEQ_LENGTH

X_train_fss = pad_sequences(X_train_fss, padding='post', maxlen=maxlen)
X_test_fss = pad_sequences(X_test_fss, padding='post', maxlen=maxlen)

y_train_fss=[x-1 for x in y_train_fss]
y_test_fss=[x-1 for x in y_test_fss]
y_train_fss = keras.utils.to_categorical(y_train_fss, num_classes=5)
y_test_fss = keras.utils.to_categorical(y_test_fss, num_classes=5)

x_train,x_test,y_train,y_test = X_train_fss[:1000,:], X_test_fss[:1000,:], y_train_fss[:1000,:],y_test_fss[:1000,:]

In [ ]:
#Yelp Full DATASET
train_data=pd.read_csv('trainyelpF.csv',header=None, names=['class_id','desc'], error_bad_lines=False, engine='python', encoding='utf8')
test_data=pd.read_csv("testyelpF.csv",header=0, names=['class_id','desc'], error_bad_lines=False, engine='python', encoding='utf8')
frames = [train_data, test_data]
dataset_agnews_fss = pd.concat(frames)

sentences_fss = dataset_agnews_fss['desc'].values
y_fss = dataset_agnews_fss['class_id'].values

sentences_train_fss, sentences_test_fss, y_train_fss, y_test_fss =train_test_split(sentences_fss, y_fss, test_size=0.25, random_state=1000)

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(sentences_train_fss)

X_train_fss = tokenizer.texts_to_sequences(sentences_train_fss)
X_test_fss = tokenizer.texts_to_sequences(sentences_test_fss)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
maxlen = MAX_SEQ_LENGTH

X_train_fss = pad_sequences(X_train_fss, padding='post', maxlen=maxlen)
X_test_fss = pad_sequences(X_test_fss, padding='post', maxlen=maxlen)

y_train_fss=[x-1 for x in y_train_fss]
y_test_fss=[x-1 for x in y_test_fss]
y_train_fss = keras.utils.to_categorical(y_train_fss, num_classes=5)
y_test_fss = keras.utils.to_categorical(y_test_fss, num_classes=5)

x_train,x_test,y_train,y_test = X_train_fss[:1000,:], X_test_fss[:1000,:], y_train_fss[:1000,:],y_test_fss[:1000,:]


In [ ]:
print('x_train shape:', x_train.shape,y_train.shape)
print('x_test shape:', x_test.shape,y_test.shape)

x_train shape: (1000, 500) (1000, 5)
x_test shape: (1000, 500) (1000, 5)


In [ ]:
def add_convpool_block(layers, max_out_ch, max_kernel_size):
    filters_lenghts = random.choice([3,4])
    deb = random.choice([2,3,4,5,6,7])
    conv_kernel = random.choice([[i for i in range(deb, deb+filters_lenghts)], [deb for i in range(filters_lenghts)]])
    
    out_channel = random.choice([64,128,512])
    #np.random.randint(128, max_out_ch)
    #conv_kernel = np.random.randint(3, max_kernel_size)

    layers.append({"type": "convpool_block", "ou_c": out_channel, "kernel": conv_kernel})

    return layers

def add_fc(layers, max_fc_neurons):
    layers.append({"type": "fc", "ou_c": np.random.randint(100, max_fc_neurons), "kernel": -1})
    
    return layers

def model_compile(list_layers,nb_classes,drop_rate,max_num_words, embedding_dim,max_seq_length):
  input_text = Input(shape=(max_seq_length,))
  embedding_layer = Embedding(max_num_words, embedding_dim, input_length=max_seq_length)(input_text)
  text_embed = SpatialDropout1D(0.2)(embedding_layer)
  filter_lengths = list_layers[0]['kernel']
  conv_layers = []
  
  for filter_length in filter_lengths:
    conv_layer = Conv1D(filters= list_layers[0]['ou_c'], kernel_size=filter_length, padding='valid',
                        strides=1, activation='relu')(text_embed)
    maxpooling = MaxPool1D(pool_size=max_seq_length - filter_length + 1)(conv_layer)
    flatten = Flatten()(maxpooling)
    conv_layers.append(flatten)
  sentence_embed = concatenate(inputs=conv_layers)
  sentence_embed = Dropout(0.5)(sentence_embed)
  dense_layer = Dense(list_layers[-1]['ou_c'], activation='relu')(sentence_embed)
  output = Dense(nb_classes, activation='softmax')(dense_layer)
  model_c = Model(input_text, output)
  model_c.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer= 'adam')
  return model_c

def model_fit(model_t,x_train, y_train, batch_size,nb_epochs, full_training = False):
  if full_training==False:
    #IMDB
    #x_train1, y_train1 = x_train[:1000], y_train[:1000]
    #x_test1, y_test1 = x_test[:1000], y_test[:1000]
    #x_train = x_train[:round(x_train.shape[0]/4)]
    #y_train = y_train[:round(y_train.shape[0]/4)]
    #Reuters and AG'News
    x_train1, y_train1 = x_train, y_train
    x_test1, y_test1 = x_test, y_test
    #print("ici1")
    #hist = model.fit(x=x_train1, y=y_train1, batch_size=batch_size, epochs=nb_epochs)
    #reuters
    #hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epochs, verbose=1, validation_split=0.1)
    hist=model_t.fit(x_train1, y_train1,epochs=1,verbose=1,batch_size=batch_size)
    #print("ici2")
    
    return model_t.evaluate(x_test1, y_test1, verbose=True)
  else:
    model_t.summary()
    hist = model_t.fit(x=x_train, y=y_train,batch_size=batch_size, epochs=nb_epochs)
    #return model.evaluate(x_test,  y_test, verbose=True)
    return model_t.evaluate(X_test_fss,  y_test_fss, verbose=True)

  #model.evaluate(x_train, y_train, verbose=True)
  



In [ ]:
import random
layers=[]
filters_lenghts = random.choice([3,4])
deb = random.choice([2,3,4,5,6,7])
conv_kernel = random.choice([[i for i in range(deb, deb+filters_lenghts)], [deb for i in range(filters_lenghts)]])
layers.append({"type": "convpool_block", "ou_c": 22, "kernel": conv_kernel})
layers[0]['kernel']

[5, 5, 5]

In [ ]:

class Fish:
  def __init__(self,max_num_words,embedding_dim,max_seq_length,max_kernel_size,nb_classes,
               min_depth, max_depth,conv_pool_prob, fc_prob, max_out_ch,max_fc_neurons,n_iter):
    # Fish Attributes
    self.delta_pos = np.nan
    self.delta_cost = 0.0
    self.weight = n_iter / 4.0
    #self.cost = np.nan
    self.has_improved = False

    # Model attributes
    self.depth = np.random.randint(min_depth, max_depth)
    self.max_num_words = max_num_words
    self.embedding_dim = embedding_dim
    self.max_seq_length = max_seq_length
    self.max_kernel_size = max_kernel_size
    self.nb_classes = nb_classes
    #self.depth = np.random.randint(min_layer, max_layer)

    self.conv_pool_prob = conv_pool_prob
    self.fc_prob = fc_prob
    self.max_fc_neurons = max_fc_neurons
    self.max_out_ch = max_out_ch
        
    self.layers = []
    self.nb_convpool_blocks=None
    self.nb_fc_layers=None
    self.acc = None
    self.loss= None
    self.model=None

    self.initialization()
  
  def __str__(self):
    string = ""
    for z in range(len(self.layers)):
      
      if self.layers[z]["type"]== "convpool_block":
        string = string + "Conv_layer" + " || " + "MaxPoling_layer" + " || "
      else:
          string = string +  "FullyConnected_layer" + " || "

    return '\033[94m'+'\033[1m'+string+'\033[0m'

    # Build fish (CNN) architecture
  def update_layers(self):
    nb_convpool_blocks= len([i for i in range(len(list_layers)) if self.layers[i]["type"] == "convpool_block"])
    nb_fc_layers= len([i for i in range(len(list_layers)) if self.layers[i]["type"] == "fc"])
    return nb_convpool_blocks,nb_fc_layers
    
  def initialization(self):
    
    self.nb_convpool_blocks = 1
    #round((self.depth * self.conv_pool_prob)/2)
    print("cnvpool=",self.nb_convpool_blocks)
    self.nb_fc_layers = 1
    #self.depth - self.nb_convpool_blocks * 2
    print("fc=",self.nb_fc_layers)
    # First block is always a convolution layer followed by a pooling layer
    for i in range(self.nb_convpool_blocks):
      self.layers = add_convpool_block(self.layers, self.max_out_ch, self.max_kernel_size)
      
    for i in range(self.nb_fc_layers):
      self.layers = add_fc(self.layers, self.max_fc_neurons)
      
    # Last layer is always a fully connected layer    
    #self.layers[-1] = {"type": "fc", "ou_c": self.nb_classes, "kernel": -1}
    
    print(self.layers)
    print("ok1")
    self.model= model_compile(self.layers,self.nb_classes,drop_rate,self.max_num_words, self.embedding_dim, self.max_seq_length)
    print("ok2")
    
    self.loss,self.acc= model_fit(self.model,x_train, y_train, batch_size,nb_epochs)
    print("ok3")
    

##### Model methods ####
  

  def model_delete(self):
    # This is used to free up memory during FSS training
    del self.model
    keras.backend.clear_session()
    tf.reset_default_graph()
    self.model = None

##### FSS methods ####

  def move_to(self,nb_convblocks_new, nb_fc_new):
    change=False
    print("Steps:",nb_convblocks_new, nb_fc_new)
    list_layers1=deepcopy(self.layers)
    if nb_convblocks_new > 0:
      change=True
      filters_lenghts = random.choice([3,4])
      deb = random.choice([2,3,4,5,6,7])
      conv_kernel = random.choice([[i for i in range(deb, deb+filters_lenghts)], [deb for i in range(filters_lenghts)]])
    
      out_channel = random.choice([64,128,512])
      #print("add convpool")
      for i in range(nb_convblocks_new):        
        list_layers1.insert(0,{"type": "convpool_block", "ou_c": out_channel, "kernel": conv_kernel})
    elif nb_convblocks_new < 0 and abs(nb_convblocks_new) < self.nb_convpool_blocks:
      #print("del convpool")
      change=True 
      for i in range(int(abs(nb_convblocks_new))):
        del list_layers1[0]
    elif nb_convblocks_new < 0 and self.nb_convpool_blocks > 1:
      change=True
      del list_layers1[:self.nb_convpool_blocks-1]
    else:
      pass;      
    if nb_fc_new > 0:
      #print("add fc")
      change=True
      for i in range(nb_fc_new):
        list_layers1.insert(-1,{"type": "fc", "ou_c": np.random.randint(1, self.max_fc_neurons), "kernel": -1})
    elif nb_fc_new < 0 and abs(nb_fc_new) < self.nb_fc_layers :
      #print("del fc")
      change=True
      print("architecture before modification:",list_layers1)
      for i in range(int(abs(nb_fc_new))):
        del list_layers1[-2]
    elif nb_fc_new < 0 and self.nb_fc_layers > 1 :
      change=True
      del list_layers1[len(list_layers1)-self.nb_fc_layers:len(list_layers1)-1]
    else:
      pass;
    return list_layers1,change

  def individual_movement(self):
    print("Starting individual movement")
    #list_layers = self.layers
    try_times=1
    max_acc = self.acc
    print("initial fish accuracy:", max_acc)
    for i in range(try_times):
      print("try number:",i)
      step_x = np.random.randint(-2,3)
      
      step_y = np.random.randint(-2,3)
      if (step_x,step_y != 0,0):
        list_layers,change = self.move_to(step_x,step_y)
        print("compiling model")
        model_ind = model_compile(list_layers,self.nb_classes, drop_rate,
                              self.max_num_words, self.embedding_dim, self.max_seq_length)
        print("training model")
        loss,acc = model_fit(model_ind,x_train, y_train, batch_size,nb_epochs)
        print("Fish accuracy after modification",acc)
        if acc > max_acc:
          print("Improvement after modification:"+'\033[92m' +"YES"+ '\033[0m')
          max_acc=acc
          self.delta_cost = acc - self.acc
          self.acc=acc
        #nb_convpool_blocks,nb_fc_layers = self.update_layers()
          self.delta_pos= (step_x ,step_y)
          self.layers= list_layers
          
          self.nb_convpool_blocks = len([x for x in list_layers if x["type"]=="convpool_block"]) 
          self.nb_fc_layers = len([x for x in list_layers if x["type"]=="fc"])
          self.loss,self.acc = loss,acc
          self.model=model_ind
          self.has_improved=True
        else: print("Improvement after modification:"+'\033[91m' +"NO"+ '\033[0m')        
    return self
        
  
  def collective_movement(self,nb_convpool_blocks,nb_fc_layers):
    list_layers,change = self.move_to(nb_convpool_blocks,nb_fc_layers)
    if change:
      self.model = model_compile(list_layers,self.nb_classes, drop_rate, self.max_num_words, self.embedding_dim, self.max_seq_length)
      loss,acc = model_fit(self.model,x_train, y_train, batch_size,nb_epochs)
      #ecrire dans le fichier
      self.delta_cost= acc - self.acc
      if self.delta_cost > 0:
        print("Improvement after collective movement:"+'\033[92m' +"YES"+ '\033[0m')
        self.has_improved= True
        self.delta_pos= (nb_convpool_blocks ,nb_fc_layers)
        self.layers= list_layers
        self.nb_convpool_blocks+=nb_convpool_blocks
        self.nb_fc_layers += nb_fc_layers
        self.loss,self.acc = loss,acc
      else:
        print("Improvement after collective movement:"+'\033[91m' +"NO"+ '\033[0m')
        self.has_improved=False

    #nb_convpool_blocks,nb_fc_layers = update_layers()
    
    
    return self


In [ ]:
class School:
  def __init__(self,n_iter,school_size):
    self.n_iter = n_iter
    self.dim=2
    self.school_size = school_size
    self.prev_weight_school = 0.0
    self.curr_weight_school = 0.0
    self.best_fish = None
    self.best_fish_acc=0.0
    self.best_fish_loss=0.0
    self.best_fish_model=None

    

    self.optimum_cost_tracking_iter = []
    self.optimum_cost_tracking_eval = []

    self.school = []
    for i in range(school_size):
      fish = Fish(max_num_words,embedding_dim,max_seq_length,max_kernel_size,nb_classes,min_depth,
                  max_depth,conv_pool_prob, fc_prob, max_out_ch,max_fc_neurons,n_iter)
      self.school.append(fish)
      
      self.curr_weight_school += fish.weight
      

    self.prev_weight_school = self.curr_weight_school
    
    self.update_best_fish()
    
     
  def update_best_fish(self):
    for fish in self.school:
      if self.best_fish_acc < fish.acc:
        self.best_fish = copy(fish)
        self.best_fish_acc = fish.acc
        self.best_fish_loss = fish.loss
        self.best_fish_model=fish.model
    
  def max_delta_cost(self):
    max_ = 0
    for fish in self.school:
      if max_ < fish.delta_cost:
        max_ = fish.delta_cost
    return max_

  def total_school_weight(self):
    self.prev_weight_school = self.curr_weight_school
    self.curr_weight_school = 0.0
    for fish in self.school:
      self.curr_weight_school += fish.acc

  def calculate_barycenter(self):
    barycenter = np.zeros((self.dim,) ,dtype=np.int) # 2 : 1 convpool blocks ; 1 fc layers
    density = 0.0
    for fish in self.school:
      density += fish.acc
      barycenter[0] += (fish.nb_convpool_blocks * fish.acc)
      barycenter[1] += (fish.nb_fc_layers * fish.acc)
    for dim in range(self.dim):
      barycenter[dim] = barycenter[dim] / density

    return barycenter

  '''def update_steps(self, curr_iter):
        self.curr_step_individual = self.step_individual_init - curr_iter * float(
            self.step_individual_init - self.step_individual_final) / self.n_iter

        self.curr_step_volitive = self.step_volitive_init - curr_iter * float(
            self.step_volitive_init - self.step_volitive_final) / self.n_iter'''
  
  
  def update_steps(self, curr_iter):
    self.curr_step_individual = self.step_individual_init - curr_iter * float(
    self.step_individual_init - self.step_individual_final) / self.n_iter
    self.curr_step_volitive = self.step_volitive_init - curr_iter * float(
    self.step_volitive_init - self.step_volitive_final) / self.n_iter
  
  
  def feeding(self):
    max_delta_cost = self.max_delta_cost()
    for fish in self.school:
      if max_delta_cost :
        fish.weight += (fish.delta_cost / max_delta_cost)
            
  '''def collective_instinctive_movement(self):
    cost_eval_enhanced = np.zeros((self.dim,), dtype=np.int)
    density = 0.0
    for fish in self.school:
      density += fish.delta_cost
      cost_eval_enhanced[0] = cost_eval_enhanced[0] + (fish.nb_convpool_blocks * fish.delta_cost)
      cost_eval_enhanced[1] = cost_eval_enhanced[1] + (fish.nb_fc_layers * fish.delta_cost)
    for dim in range(self.dim):
      if density != 0:
        cost_eval_enhanced[dim] = cost_eval_enhanced[dim] / density
    for fish in self.school:
      nb_convpool_blocks = int(round(cost_eval_enhanced[0]))      
      nb_fc_layers = int(round(cost_eval_enhanced[1]))
      #if nb_convpool_blocks + nb_fc_blocks > max_depth or nb_convpool_blocks + nb_fc_blocks < min_depth:
        #continue;
      #else:
      print("Collective_instinctive_movement:",nb_convpool_blocks,nb_fc_layers)
      if (nb_convpool_blocks,nb_fc_layers) != (0,0):
        fish.collective_movement(nb_convpool_blocks,nb_fc_layers)
      else:
        print("null instinctive ")'''
  def collective_movement(self):
    def sortThird(val): 
      return val[2] 
    self.total_school_weight()
    list1 =  [(fish.nb_convpool_blocks,fish.nb_fc_layers,fish.acc) for fish in self.school ]
    list1.sort(key = sortThird, reverse = True)    
    averge_convpool_blocks = np.mean([list1[i][0] for i in range (len(list1)//2)])
    print("Average convpool",averge_convpool_blocks) 
    average_fc_layers = np.mean([list1[i][1] for i in range (len(list1)//2)]) 
    print("Average fc",average_fc_layers) 
    for fish in self.school:
      #ecrire dans le fichier
      if self.curr_weight_school > self.prev_weight_school:
        if fish.nb_convpool_blocks > averge_convpool_blocks:
          nb_convpool_blocks = -1 * abs((averge_convpool_blocks - fish.nb_convpool_blocks)) * 0.5 
        else:
          nb_convpool_blocks = abs((averge_convpool_blocks - fish.nb_convpool_blocks)) * 0.5 
        if fish.nb_fc_layers > average_fc_layers:
          nb_fc_layers =  -1 * abs((average_fc_layers - fish.nb_fc_layers)) * 0.5 
        else:
          nb_fc_layers =  abs((average_fc_layers - fish.nb_fc_layers)) * 0.5 
      else:
        if fish.nb_convpool_blocks > averge_convpool_blocks:
          nb_convpool_blocks = abs((averge_convpool_blocks - fish.nb_convpool_blocks)) * 0.5 
        else:
          nb_convpool_blocks = -1* abs((averge_convpool_blocks - fish.nb_convpool_blocks)) * 0.5 
        if fish.nb_fc_layers > average_fc_layers:
          nb_fc_layers =  abs((average_fc_layers - fish.nb_fc_layers)) * 0.5 
        else:
          nb_fc_layers = -1 * abs((average_fc_layers - fish.nb_fc_layers)) * 0.5 
      nb_convpool_blocks,nb_fc_layers = int(nb_convpool_blocks),int(nb_fc_layers)
      '''nb_convpool_blocks = round((averge_convpool_blocks -  fish.nb_convpool_blocks)/2)     
      nb_fc_layers = round((average_fc_layers - fish.nb_fc_layers) / 2 )
      #if nb_convpool_blocks + nb_fc_blocks > max_depth or nb_convpool_blocks + nb_fc_blocks < min_depth:
        #continue;
      #else:
      if (nb_convpool_blocks,nb_fc_layers)!=(0,0):
        fish.collective_movement(int(nb_convpool_blocks),int(nb_fc_layers))
      else:
        print("NULL instinctive_movement")
        #print("OKKKKKKKKK",nb_convpool_blocks,nb_fc_layers)
      #else:
        #print("one of the dimension is negative!",nb_convpool_blocks,nb_fc_layers)'''
      if (nb_convpool_blocks,nb_fc_layers)!=(0,0):
        print('\033[95m'+"collective movement steps:"+'\033[0m',int(nb_convpool_blocks),int(nb_fc_layers))
        fish.collective_movement(int(nb_convpool_blocks),int(nb_fc_layers))
      else:
        print('\033[91m'+"null collective movement"+'\033[0m')
  '''def collective_volitive_movement(self):
    
    self.total_school_weight()
    barycenter = self.calculate_barycenter()
    for fish in self.school:
      if self.curr_weight_school > self.prev_weight_school:
        if fish.nb_convpool_blocks > barycenter[0]:
          nb_convpool_blocks = -1 * abs((barycenter[0] - fish.nb_convpool_blocks)) * 0.5 
        else:
          nb_convpool_blocks = abs((barycenter[0] - fish.nb_convpool_blocks)) * 0.5 
        if fish.nb_fc_layers > barycenter[1]:
          nb_fc_layers =  -1 * abs((barycenter[1] - fish.nb_fc_layers)) * 0.5 
        else:
          nb_fc_layers =  abs((barycenter[1] - fish.nb_fc_layers)) * 0.5 
      else:
        if fish.nb_convpool_blocks > barycenter[0]:
          nb_convpool_blocks = abs((barycenter[0] - fish.nb_convpool_blocks)) * 0.5 
        else:
          nb_convpool_blocks = -1* abs((barycenter[0] - fish.nb_convpool_blocks)) * 0.5 
        if fish.nb_fc_layers > barycenter[1]:
          nb_fc_layers =  abs((barycenter[1] - fish.nb_fc_layers)) * 0.5 
        else:
          nb_fc_layers = -1 * abs((barycenter[1] - fish.nb_fc_layers)) * 0.5 
      nb_convpool_blocks,nb_fc_layers = int(nb_convpool_blocks),int(nb_fc_layers)
      print("Collective_volitive_movement:",nb_convpool_blocks,nb_fc_layers)
      if (nb_convpool_blocks,nb_fc_layers)!=(0,0):
        fish.collective_movement(int(nb_convpool_blocks),int(nb_fc_layers))
      else:
        print("null volitive")
      #cost = self.objective_function.evaluate(new_pos)
    #self.update_best_fish()'''

  def optimize(self):
    #self.__init__(max_num_words,embedding_dim,max_seq_length,max_kernel_size,nb_classes,
               #min_depth, max_depth,conv_pool_prob, fc_prob, max_out_ch,max_fc_neurons,n_iter,school_size)
    track_Best_fish=[]
    track_fish_before_indiv_mvt=[]
    track_fish_after_indiv_mvt=[]
    track_fish_before_collec_mvt=[]
    track_fish_before_collec_mvt=[]
    
    for i in tqdm(range(self.n_iter)):
      find.write("Iteration number: " + str(i)+'\n')
      fcoll.write("Iteration number: " + str(i)+'\n')
      fiter.write("Iteration number: " + str(i)+'\n')
      k=1
      for fish in self.school:
        find.write("fish number "+str(k)+'before individual movement :'+str(fish.acc)+'---')
        fish.individual_movement()
        find.write('after individual movement :'+str(fish.acc)+'\n')
        k+=1
      fiter.write('Best fish before individual movement:'+ str(self.best_fish_acc)+'----')
      self.update_best_fish()
      fiter.write('Best fish after individual movement:'+ str(self.best_fish_acc)+'\n')
      #self.feeding()
      fcoll.write('Best fish before collective movement:'+ str(self.best_fish_acc)+'----')
      self.collective_movement()
      #self.collective_volitive_movement()
      #self.update_steps(i)
      self.update_best_fish()
      fcoll.write('Best fish after collective movement:'+ str(self.best_fish_acc)+'\n')
      
      print ("Iteration: ", i, " Cost: ", self.best_fish.acc)
    

In [ ]:
def create_dir(path):
    directory = os.path.dirname(path)
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)

In [ ]:
max_num_words= MAX_NUM_WORDS
embedding_dim=EMBEDDING_DIM
max_seq_length=MAX_SEQ_LENGTH
max_kernel_size=7
nb_classes=10

min_depth=2
max_depth=3
# Probability of each layer type (should sum to 1)
conv_pool_prob=0.5
fc_prob=0.5

max_out_ch= 256
max_fc_neurons= 300

drop_rate=0.5
batch_size=BATCH_SIZE
nb_epochs = NB_EPOCHS
epochs_full_training = 1

runs_time=[]
# FSS parameters
results_path = "Executions/"
num_runs = 1
school_size = 20
num_iterations = 2
all_gbest_par=[]
create_dir(results_path)
find=open(results_path+'/individual_movement.txt', 'w')
find.write("individual movement track\n")
fcoll=open(results_path+'/collective_movement.txt', 'w')
fcoll.write("collective movement track\n")
fbest=open(results_path+'/Best_fish.txt', 'w')
fbest.write("global best fish track\n")
fiter=open(results_path+'/best_fish_OverIterations.txt', 'w')
fiter.write("Best fish Over Iterations track\n")
      
for i in range (num_runs):
  print("******************Run number: " + str(i)+'************************\n')
  fbest.write("******************Run number: " + str(i)+'************************\n')
  find.write("******************Run number: " + str(i)+'************************\n')
  fcoll.write("******************Run number: " + str(i)+'************************\n')
  fiter.write("******************Run number: " + str(i)+'************************\n')
  start_time = time.time()
  s = School(num_iterations,school_size)
  s.optimize()
  fbest.write("Best fish accuracy: " + str(s.best_fish_acc)+'\n')
  print(s.best_fish_acc)
  # Plot current Best_fish
  matplotlib.use('Agg')
  plt.plot(s.best_fish_acc)
  plt.xlabel("Run")
  plt.ylabel("Fish Best accuracy")
  plt.savefig(results_path + "gBest-iter-" + str(i) + ".png")
  plt.close()

  print('Best architecture found: ')
  print(s.best_fish)
  fbest.write("Best fish architecture found: " + s.best_fish.__str__() +'\n')
  
  #fbest.write("Parameters of Best fish architecture found: " + s.best_fish.layer'\n')  
  np.save(results_path + "Best_inter_" + str(i) + "_acc_history.npy", s.best_fish_acc)

  #np.save(results_path + "gBest_iter_" + str(i) + "_test_acc_history.npy", s.gBest_test_acc)

  end_time = time.time()

  running_time = end_time - start_time
  fbest.write("Running Time: " + str(running_time) +'\n')
  runs_time.append(running_time)

  # Fully train and evaluate the Best model found
   
  n_parameters = model_fit(s.best_fish.model,X_train_fss, y_train_fss, batch_size,epochs_full_training, full_training = True)
  #n_parameters = model_fit(s.best_fish.model,x_train, y_train, batch_size,epochs_full_training, full_training = True)
 
  all_gbest_par.append(n_parameters)
  print(all_gbest_par)
  fbest.write("final model evaluation: " + '\n')
  l1=map(lambda x:str(x)+';', all_gbest_par)
  fbest.writelines(l1)
  para=[]
  for nn in s.best_fish.layers:
    para=para+list(nn.values())
  l1=map(lambda x:str(x)+';', para)
  fbest.writelines(l1)
  # Evaluate the fully trained gBest model
   #gBest_metrics = pso.evaluate_gBest(batch_size=batch_size_full_training)
find.close()
fcoll.close()
fiter.close()
fbest.close()
        




******************Run number: 0************************

cnvpool= 1
fc= 1
[{'type': 'convpool_block', 'ou_c': 512, 'kernel': [3, 3, 3]}, {'type': 'fc', 'ou_c': 169, 'kernel': -1}]
ok1
ok2


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
1000/1000 [==============================] - 0s 465us/step
ok3
cnvpool= 1
fc= 1
[{'type': 'convpool_block', 'ou_c': 64, 'kernel': [2, 3, 4, 5]}, {'type': 'fc', 'ou_c': 111, 'kernel': -1}]
ok1
ok2
Epoch 1/1
1000/1000 [==============================] - 0s 314us/step
ok3
cnvpool= 1
fc= 1
[{'type': 'convpool_block', 'ou_c': 128, 'kernel': [2, 3, 4]}, {'type': 'fc', 'ou_c': 127, 'kernel': -1}]
ok1
ok2
Epoch 1/1
1000/1000 [==============================] - 0s 305us/step
ok3
cnvpool= 1
fc= 1
[{'type': 'convpool_block', 'ou_c': 128, 'kernel': [5, 5, 5, 5]}, {'type': 'fc', 'ou_c': 214, 'kernel': -1}]
ok1
ok2
Epoch 1/1
1000/1000 [==============================] - 0s 420us/step
ok3
cnvpool= 1
fc= 1
[{'type': 'convpool_block', 'ou_c': 512, 'kernel': [2, 3, 4]}, {'type': 'fc', 'ou_c': 150, 'kernel': -1}]
ok1
ok2
Epoch 1/1
1000/1000 [==============================] - 0s 490us/step
ok3
cnvpool= 1
fc= 1
[{'type': 'convpool_block', 'ou_c': 512, 'kernel': [5, 6, 7]}, {'type': 'fc', 'ou_c': 111

  0%|          | 0/2 [00:00<?, ?it/s]

ok3
Starting individual movement
initial fish accuracy: 0.18299999833106995
try number: 0
Steps: -2 1
compiling model
training model
Epoch 1/1
1000/1000 [==============================] - 0s 475us/step
Fish accuracy after modification 0.11999999731779099
Improvement after modification:NO
Starting individual movement
initial fish accuracy: 0.10499999672174454
try number: 0
Steps: 1 1
compiling model
training model
Epoch 1/1
1000/1000 [==============================] - 1s 736us/step
Fish accuracy after modification 0.11100000143051147
Improvement after modification:YES
Starting individual movement
initial fish accuracy: 0.11999999731779099
try number: 0
Steps: 1 -2
compiling model
training model
Epoch 1/1
1000/1000 [==============================] - 1s 524us/step
Fish accuracy after modification 0.10499999672174454
Improvement after modification:NO
Starting individual movement
initial fish accuracy: 0.11100000143051147
try number: 0
Steps: -1 0
compiling model
training model
Epoch 1/1
10

 50%|█████     | 1/2 [01:09<01:09, 69.16s/it]

Fish accuracy after modification 0.10999999940395355
Improvement after modification:NO
Average convpool 1.3
Average fc 1.1
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
Iteration:  0  Cost:  0.18299999833106995
Starting individual movement
initial fish accuracy: 0.18299999833106995
try number: 0
Steps: 1 -1
compiling model
training model
Epoch 1/1
1000/1000 [==============================] - 0s 301us/step
Fish accuracy after modification 0.1080000028014183
Improvement after modification:NO
Starting individual movement
initial fish accuracy:

100%|██████████| 2/2 [02:12<00:00, 66.31s/it]

Fish accuracy after modification 0.10000000149011612
Improvement after modification:NO
Average convpool 1.5
Average fc 1.1
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
Iteration:  1  Cost:  0.18299999833106995
0.18299999833106995
Best architecture found: 
Conv_layer || MaxPoling_layer || FullyConnected_layer || 
Model: "model_123"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_123 (Inp

62572/62572 [==============================] - 26s 409us/step
[[1.323615030262969, 0.5628236532211304]]


In [ ]:
s.best_fish.model.summary()

Model: "model_187"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_187 (InputLayer)          (None, 512)          0                                            
__________________________________________________________________________________________________
embedding_187 (Embedding)       (None, 512, 300)     1500000     input_187[0][0]                  
__________________________________________________________________________________________________
spatial_dropout1d_187 (SpatialD (None, 512, 300)     0           embedding_187[0][0]              
__________________________________________________________________________________________________
conv1d_667 (Conv1D)             (None, 508, 512)     768512      spatial_dropout1d_187[0][0]      
__________________________________________________________________________________________

In [ ]:
hist = s.best_fish.model.fit(x=x_train, y=y_train,batch_size=1, epochs=10)

Epoch 1/10
1000/1000 [==============================] - 9s 9ms/step - loss: 1.0551 - acc: 0.5760
Epoch 2/10
1000/1000 [==============================] - 9s 9ms/step - loss: 0.6431 - acc: 0.7440
Epoch 3/10
1000/1000 [==============================] - 9s 9ms/step - loss: 0.4348 - acc: 0.8360
Epoch 4/10
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3766 - acc: 0.8800
Epoch 5/10
1000/1000 [==============================] - 9s 9ms/step - loss: 0.2207 - acc: 0.9350
Epoch 6/10
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1418 - acc: 0.9560
Epoch 7/10
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1401 - acc: 0.9540
Epoch 8/10
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1088 - acc: 0.9650
Epoch 9/10
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1105 - acc: 0.9630
Epoch 10/10
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1872 - acc: 0.9520


In [ ]:
result=s.best_fish.model.evaluate(x_test,  y_test, verbose=True)

1000/1000 [==============================] - 1s 517us/step


In [ ]:
print(hist.history.keys())

dict_keys(['loss', 'acc'])


In [ ]:
plt.plot(hist.history['acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')

plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)
from sklearn import model_selection

In [ ]:
hist.history['acc']

[0.576, 0.744, 0.836, 0.88, 0.935, 0.956, 0.954, 0.965, 0.963, 0.952]

In [ ]:
# Compare Algorithms

# evaluate each model in turn
results = [hist.history['acc']]
names = ['bbbb']

# boxplot algorithm comparison
plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
#ax.set_xticklabels(names)
plt.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
acc_train = hist.history['acc']
loss_train = hist.history['loss']

#loss_val = history.history['val_acc']
epochs = range(1,11)
plt.plot(epochs, acc_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_train, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(n_parameters)
ax.set_xticklabels(['ff'])
plt.show()
